# Введение в глубинное обучение, ФКН ВШЭ

# Практическое задание 2. Рекуррентные Нейронные Сети

## Общая информация
Дата выдачи: 20.02.2021

Мягкий дедлайн: 15.03.2021 23:59 MSK

Жёсткий дедлайн: 18.03.2021 00:59 MSK

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Система оценивания работы находится в конце этого ноутбука.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).


## Формат сдачи
Задания сдаются семинаристу на почту. Посылка должна содержать:
* Ноутбук homework-practice-02-Username.ipynb

Username — ваша фамилия на латинице

Тема письма: #группы ДЗ2 ФИ

### Ссылка на соревнование Kaggle и данные: https://www.kaggle.com/c/nlp-intro-to-dl-2021/overview

## 0. Подготовка данных

Данные представляют собой корпус текстов с 4-мя категориями. Ваша задача - написать классификатор для этих данных, определяющий, к какой из категорий относится текст.

In [53]:
import pandas as pd
df_train = pd.read_csv("./train.csv")
df_train.head()

,text,source
0,Российская сборная лидирует по итогам командно...,mchsgov
1,#СоветМЧС #МЧС #МЧСРОССИИ,mchsgov
2,Инспекторы ГИБДД Москвы приняли участие во Все...,mospolice
3,В Главную Военно-Морскую Базу БФ в г.Балтийск ...,mil
4,Обвиняемые в хищении денежных средств у 32 пож...,mospolice


In [156]:
df_test = pd.read_csv("./test.csv")
df_test.head()

,text
0,На полигоне «Погоново» Воронежской области про...
1,#БудниМЧС #МЧС #МЧСРОССИИ <br><br>Пожарные тра...
2,"Друзья, напоминаем вам, что завтра единый день..."
3,"Настольная игра: ""Королевская почта""<br><br>По..."
4,Миллиарды писем разносят по почтовым ящикам на...


## 1. Предобработка данных (1 балл)

В этом задании вам предстоит провести предобработку данных. Баллы ставятся следующим образом:

* Привести все тексты к одной длине, заменить слова/токены на числа, факторизовать целевую переменную и т.д.

* Использовать токенизатор, который разбил бы все слова на токены (подробнее https://github.com/huggingface/tokenizers).

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
v = CountVectorizer(max_features=300)
v.fit(pd.concat([df_train["text"], df_test["text"]]).values.astype('U'))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=300, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [23]:
labeling = LabelEncoder()

In [24]:
X = v.transform(df_train["text"].values.astype('U')).toarray()

In [25]:
X_global = v.transform(df_test["text"].values.astype('U')).toarray()

In [26]:
df_train["source"].value_counts()

mchsgov        2575
mil            2575
mospolice      2474
russianpost    2257
Name: source, dtype: int64

In [27]:
labeling = LabelEncoder()
y = labeling.fit_transform(df_train["source"])

## 2. LSTM-сеть (4 балла)

В этом задании вам предстоит написать LSTM сеть __вручную__ (то есть без использования стандартных реализаций из keras / torch / tensorflow). Сама архитектура отлично расписана здесь: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 28kB/s 
     |████████████████████████████████| 8.8MB 23.3MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

In [29]:
class LSTM_cell(nn.Module):
    def __init__(self, input_size, hidden_size):
        # Base parameters
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Forget gate layer
        self.W_f = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_f = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(hidden_size))

        # Input gate layer
        self.W_i = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_i = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(hidden_size))

        # Candidate values for a cell
        self.W_c = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_c = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(hidden_size))

        # Output gate layer
        self.W_o = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.U_o = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(hidden_size))

        self.initial_weights()

    def initial_weights(self):
        std = 1 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-std, std)

    def forward(self, x, initial_states=None):
        batch_size, total_cells, _ = x.size()
        hidden_sequence = []

        if initial_states is None:
            h_t, c_t = (
                torch.zeros(batch_size, self.hidden_size).to(x.device),
                torch.zeros(batch_size, self.hidden_size).to(x.device),
            )
        else:
            h_t, c_t = initial_states

        for state in range(total_cells):
            x_t = x[:, state, :]

            # Calculate gates
            f_t = torch.sigmoid(h_t @ self.U_f + x_t @ self.W_f + self.b_f)
            i_t = torch.sigmoid(h_t @ self.U_i + x_t @ self.W_i + self.b_i)
            cv_t = torch.tanh(h_t @ self.U_c + x_t @ self.W_c + self.b_c)
            c_t = c_t * f_t + i_t * cv_t

            # Calculate output gate layer and hidden state
            o_t = torch.sigmoid(h_t @ self.U_o + x_t @ self.W_o + self.b_o)
            h_t = o_t * torch.tanh(c_t)

            # Add output h_n to the sequence
            hidden_sequence.append(h_t.unsqueeze(0))

        # Transform hidden sequence
        hidden_sequence = torch.cat(hidden_sequence, dim=0)
        hidden_sequence = hidden_sequence.transpose(0, 1).contiguous()
        return hidden_sequence, (h_t, c_t)

## 3. Модель

В этом задании вам предстоит объединить вашу сеть с несколькими другими слоями для создания итоговой модели классификатора (можно начать с самой базовой архитектуры, слой эмбеддингов - LSTM - выходной слой).

In [44]:
class LSTM_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(300, 32)
        self.LSTM = LSTM_cell(32, 32)
        self.fc = nn.Linear(32, 4)

    def forward(self, x):
        x = self.embedding(x)
        x, (h_n, c_n) = self.LSTM(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

## 4. Обучение модели.

Обученную модель примените для получения предсказаний на Kaggle. Баллы за задания из ноутбука (максимум 5) ставятся в случае, если модель корректно отработала, и вы смогли успешно отправить посылку на Kaggle.

In [7]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=808
)

X_train, y_train = torch.tensor(X_train), torch.tensor(y_train)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

In [47]:
model_train = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(
    model_train, batch_size=128, shuffle=True
)

model_test = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(
    model_test, batch_size=128, shuffle=True
)

In [48]:
device = torch.device("cuda")
clf = LSTM_model().to(device)
optimizer = optim.Adam(clf.parameters(), lr=0.004)
criterion = nn.CrossEntropyLoss()

In [49]:
epoch_bar = tqdm(range(15), desc="Training", position=0, total=2)
acc = 0

for epoch in epoch_bar:
    batch_bar = tqdm(
        enumerate(train_loader),
        desc="Epoch: {}".format(str(epoch)),
        position=0,
        total=len(train_loader),
    )

    for i, (datapoints, labels) in batch_bar:
        optimizer.zero_grad()

        preds = clf(datapoints.long().to(device))
        loss = criterion(preds, labels.to(device))
        loss.backward()
        optimizer.step()

        if (i + 1) % 50 == 0:
            acc = 0

            with torch.no_grad():
                for i, (datapoints_, labels_) in enumerate(test_loader):
                    preds = clf(datapoints_.to(device))
                    acc += (
                        (preds.argmax(dim=1) == labels_.to(device))
                        .float()
                        .sum()
                        .cuda()
                        .item()
                    )
            acc /= len(X_test)

        batch_bar.set_postfix(
            loss=loss.cuda().item(), accuracy="{:.2f}".format(acc), epoch=epoch
        )
        batch_bar.update()

    epoch_bar.set_postfix(
        loss=loss.cuda().item(), accuracy="{:.2f}".format(acc), epoch=epoch
    )
    epoch_bar.update()

Epoch: 14: 100%|██████████| 62/62 [00:23<00:00,  2.69it/s, accuracy=0.64, epoch=14, loss=0.877]
Training: 15it [05:47, 23.18s/it, accuracy=0.64, epoch=14, loss=0.877]


In [118]:
X_global = torch.tensor(X_global)
model_global = torch.utils.data.TensorDataset(X_global)
global_loader = torch.utils.data.DataLoader(
    model_global, batch_size=64
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [5]:
import numpy as np

In [131]:
preds = clf(X_global.to(device))

In [135]:
preds = preds.cpu().argmax(dim=1).numpy().tolist()

In [139]:
sub_preds = labeling.inverse_transform(preds)

In [157]:
df_test["Category"] = sub_preds

In [143]:
submission = df_test["Category"]

In [158]:
df_test = df_test.reset_index()

In [161]:
submission = df_test[["index", "Category"]]
submission.columns = ["id", "Category"]

In [165]:
submission.to_csv("submission1.csv", index=0)

## Система оценивания домашнего задания

В домашнем задании две смысловые части:

1. выполнение заданий из ноутбука --- **5 баллов**

2. создание архитектуры сети, дающей как можно лучшее качество на Kaggle --- **5 баллов (+5 бонусных баллов)**. Эта архитектура не обязана содержать написанный вами собственноручно LSTM слой.


#### Оценивание результатов Kaggle:

Основные баллы (максимум 5) вычисляются по следующему правилу по public leaderboard:

val_accuracy $\geq$ baseline (0.96367): 1 балл

val_accuracy $\geq$ 0.968: 2 балла

val_accuracy $\geq$ 0.972: 3 балла

val_accuracy $\geq$ 0.976: 4 балла

val_accuracy $\geq$ 0.98: 5 баллов

Бонусные баллы:

Если вы верно выполнили задание с имплементацией LSTM в этом ноутбуке и попали в топ-3 **среди всех студентов, посещающих курс**, вы получаете бонусные 5 баллов.

**Максимальный суммарный балл за домашнее задание** - 15.

In [1]:
import pandas as pd
df_train = pd.read_csv("./train.csv")
df_train.head()

df_test = pd.read_csv("./test.csv")
df_test.head()

,text
0,На полигоне «Погоново» Воронежской области про...
1,#БудниМЧС #МЧС #МЧСРОССИИ <br><br>Пожарные тра...
2,"Друзья, напоминаем вам, что завтра единый день..."
3,"Настольная игра: ""Королевская почта""<br><br>По..."
4,Миллиарды писем разносят по почтовым ящикам на...


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)

In [4]:
v = TfidfVectorizer(preprocessor=preprocessing)

In [5]:
df_train["text"] = df_train["text"].fillna(" ")

In [6]:
v = TfidfVectorizer()
v.fit(df_train["text"].values)

X = v.transform(df_train["text"].values)
X = v.transform(df_train["text"].values)

X_global = v.transform(df_test["text"].values.astype('U'))

In [8]:
labeling = LabelEncoder()
y = labeling.fit_transform(df_train["source"])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
import xgboost as xgb

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=808, stratify=y,
)

In [39]:
xgb_cool3 = xgb.XGBClassifier(colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.18, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=140, nthread=-1,
       objective='multi:softmax', reg_alpha=0, reg_lambda=3,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [51]:
xgb_cool3.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.18, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=-1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              silent=True, subsample=1, verbosity=1)

In [18]:
for1 = RandomForestClassifier(max_depth=100, n_estimators=32)
for2 = RandomForestClassifier(max_depth=None, n_estimators=50)
for3 = RandomForestClassifier(max_depth=None, n_estimators=20)

In [14]:
from sklearn.metrics import accuracy_score

In [52]:
accuracy_score(y_test, xgb_cool3.predict(X_test))

0.9595278246205734

In [16]:
from sklearn.ensemble import StackingClassifier

In [19]:
estimators = [
    ('rf', for2),
    ('svr', xgb_cool),
    ('svr21', LogisticRegression(C=50, max_iter=1000)),
    ('svr2', for3)
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(C=40, max_iter=1000)
)

In [43]:
clf.fit(X, y)

StackingClassifier(cv=None,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                     

In [44]:
preds = clf.predict(X_global)

In [45]:
sub_preds = labeling.inverse_transform(preds)

In [46]:
df_test["Category"] = sub_preds
df_test = df_test.reset_index()
submission = df_test[["index", "Category"]]
submission.columns = ["id", "Category"]
submission.to_csv("submission13.csv", index=0)